In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#%tensorflow_version 2.x  # this line is not required unless you are in a notebook
import tensorflow as tf
print(tf.version.VERSION)    
from tensorflow.keras import datasets, layers, models
import IPython


# Data

In [ ]:
# imports

from ecg_plotting import *
# from IPython.core.display_functions import display
import numpy as np
# download datasets

from dataset_downloader import download_dataset
import datasets.ptb_xl.data_handling as ptb_xl_dh

download_dataset('backgrounds')
download_dataset('ptb_xl')

## Processing and Labeling

In [ ]:
import os
os.getcwd()

In [ ]:
from datasets.ptb_xl.data_handling import get_ecg_array
Nsamples = 100
X, Ylabel = get_ecg_array(sampling_rate=500,max_samples=Nsamples)
X = np.reshape(X[:,452:-452,:],(-1, 4096,12 ,1))
Y = np.zeros((Nsamples))

dangerList = ["IMI","ASMI","ILMI","AMI","LMI","IPLMI","IPMI","PMI"]
for k in range(Nsamples):
    if len(set(Ylabel['scp_codes'].iloc[k].keys()) & set(dangerList)):
        Y[k] = 1


# Network definition

In [ ]:
class Residual(layers.Layer):
    def __init__(self, LastNumFilters,**kwargs):
        super(Residual, self).__init__(**kwargs)
        self.LastNumFilters = LastNumFilters
        
        self.layer_1 = layers.Conv2D(LastNumFilters+64, (16, 1), activation=None, padding='same')
        self.layer_2 = layers.MaxPooling2D((2, 1)) # CHECK BECAUSE PAPER DOESN'T MENTION PRECISELY
        self.layer_3 = layers.BatchNormalization(axis=[1,2]) #? axis = [1,2] to normalize the axis=0 (over the batch)
        self.layer_4 = layers.Activation(activation='relu')
        self.layer_5 = layers.Dropout(dropOutP) # To prevent overfit
        self.layer_6 = layers.Conv2D(LastNumFilters+64, (16, 1), activation=None, padding='same')
        self.layer_7 = layers.MaxPooling2D((2, 1)) # CHECK BECAUSE PAPER DOESN'T MENTION PRECISELY
        self.layer_8 = layers.BatchNormalization(axis=[1,2]) #? axis = [1,2] to normalize the axis=0 (over the batch)
        self.layer_9 = layers.Activation(activation='relu')
        self.layer_10 = layers.Dropout(dropOutP) # To prevent overfit

        self.layer_11 = layers.Conv2D(LastNumFilters+64, (1, 1), activation=None, padding='same')
        self.layer_12 = layers.MaxPooling2D((4, 1)) # CHECK BECAUSE PAPER DOESN'T MENTION PRECISELY
        self.layer_13 = layers.BatchNormalization(axis=[1,2]) #? axis = [1,2] to normalize the axis=0 (over the batch)
        self.layer_14 = layers.Activation(activation='relu')
        self.layer_15 = layers.Dropout(dropOutP) # To prevent overfit
        


    def call(self, x):
        # the residual block using Keras functional API
        xbackup = x
        LastNumFilters = self.LastNumFilters

        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)
        x = self.layer_6(x)
        x = self.layer_7(x)
        x = self.layer_8(x)
        x = self.layer_9(x)
        x = self.layer_10(x)

        xbackup = self.layer_11(xbackup)
        xbackup = self.layer_12(xbackup)
        xbackup = self.layer_13(xbackup)
        xbackup = self.layer_14(xbackup)                                                           
        xbackup = self.layer_15(xbackup)
        
        x = layers.Add()([x,xbackup])

        return x


In [ ]:
#TODO: Construct the model
dropOutP = 0.1

model = models.Sequential()

model.add(layers.Conv2D(64, (16, 1), activation=None, input_shape=(4096, 12, 1)))
model.add(layers.BatchNormalization(axis=[1,2])) #? axis = [1,2] to normalize the axis=0 (over the batch)
model.add(layers.Activation(activation='relu'))
model.add(layers.Dropout(dropOutP)) # To prevent overfit

lastNumFilters = 64
model.add(Residual(lastNumFilters))
lastNumFilters += 64
model.add(Residual(lastNumFilters))
lastNumFilters += 64
model.add(Residual(lastNumFilters))
lastNumFilters += 64
model.add(Residual(lastNumFilters))
lastNumFilters += 64

model.add(layers.Flatten())

model.add(layers.Dense(1,activation='sigmoid'))

model.summary()

## Compile

In [ ]:
#TODO: Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


## Train

In [ ]:
Y.shape

In [ ]:
model.fit(X, Y, epochs=100, batch_size=2)
